In [1]:
train_path = "data/00--raw/football-field-detection.v15i.yolov5pytorch/train/"

IMG_PATH = f"{train_path}images/0a2d9b_2_3_png.rf.2b39030ff9f2e93a34aa9ca69abbd77c.jpg"
LBL_PATH = f"{train_path}/labels/0a2d9b_2_3_png.rf.2b39030ff9f2e93a34aa9ca69abbd77c.txt"

IMG_PATH = f"{train_path}images/0a2d9b_6_11_png.rf.2cfd6b6dad39a0f39eee5eb3d729823f.jpg"
LBL_PATH = f"{train_path}/labels/0a2d9b_6_11_png.rf.2cfd6b6dad39a0f39eee5eb3d729823f.txt"


In [ ]:
import cv2
import numpy as np
from src.struct.shared_data   import SharedAnnotations
from src.visual.field         import FieldVisualizer, PitchConfig
from src.visual.visualizer    import Visualizer

# 1) Roboflow → your index mapping
rf2my = {
     1:  0,    2:  7,   
    11: 27, 12: 28,
    13: 10,   4: 16,   
     8: 18,  9:  5,
     7: 17,  15: 23,   
    16: 25, 17: 26,
    10:  9,  26: 22,   
    27:  1, 28: 11,
    20: 13,  21: 29,   
    22: 30, 23: 14,
    24:  6,  29: 15,   
    25: 21
}

rf2my2 = {
    1:   0,
    2:   7,
    3:  15,
    4:  16,
    5:   8,
    6:   2,
    7:  17,
    8:  18,
    9:   5,
    10:  9,
    11: 27,
    12: 28,
    13: 10,
    14:   -1,
    15: 23, 
    16: 25,
    17: 26,
    18:   -1,
    19:   -1,
    20: 13, 
    21: 29,
    22: 30, 
    23: 14,
    24:  6,
    25: 21,
    26: 22,   
    27:  1, 
    28: 11,
    29: 15,   
}

# class  xc  yc  w  h   x0  y0  v0   x1  y1  v1   x2  y2  v2 ... xN yN vN

# field     count       meaning                                     coordinate space
# class	    1	        always 0 (only one class: “football field”)	–
# xc yc w h	4	        YOLO bounding-box (we normally ignore it)	normalised (0-1)
# xi yi	    2 × (N+1)	position of landmark i	                    normalised (0-1)
# vi	    1 × (N+1)	visibility flag (0 = not labelled / occluded, 1 = labelled)


# 2) load one example
IMG = IMG_PATH
LBL = LBL_PATH

img = cv2.imread(IMG)
h, w = img.shape[:2]

# 3) parse Roboflow keypoints
vals = list(map(float, open(LBL).read().split()))
kp   = np.array(vals[5:], dtype=np.float32).reshape(-1,3)  # (num_pts, 3)

# 4) init shared‐data and field
shared    = SharedAnnotations()
fv        = FieldVisualizer(PitchConfig(draw_points=True))
model_pts = fv.get_hardcoded_model_points()  # shape (M,2)

# 5) build correspondences
pts_img   = []
pts_model = []
for j, (xn,yn,vis) in enumerate(kp):
    if vis <= 0 or j not in rf2my:
        continue
    my_idx = rf2my[j]

    # image‐space pixel
    x_px, y_px = float(xn*w), float(yn*h)
    shared.captured_video_pts.append((x_px, y_px))
    pts_img.append([x_px, y_px])

    # model‐space metre
    mx, my = model_pts[my_idx]
    shared.ground_truth_pts.append((mx, my))
    pts_model.append([mx, my])

pts_img   = np.array(pts_img,   dtype=np.float32)
pts_model = np.array(pts_model, dtype=np.float32)

if len(pts_img) < 4:
    raise RuntimeError(f"Need ≥4 matches, got {len(pts_img)}")

# 6) fit homography H: model→image
H, _                 = cv2.findHomography(pts_model, pts_img, cv2.RANSAC, 3.0)
shared.H_video2field = H

# 7) set up the Visualizer
class DummyProcess:
    def __init__(self, sd):      self.shared_data = sd
    def is_done(self):           return True
    def on_mouse_click(self, x,y): pass

viz = Visualizer(
    field_config = PitchConfig(linewidth=2, draw_points=True),
    frame        = img,             # initial video frame
    class_names  = {}, 
    process      = DummyProcess(shared)
)

# prime the video side
viz.video_visualizer.update(type("VF",(object,),{
    "frame_id":0, "timestamp":0.0,
    "image":img,
    "detections":[]
})())

# ─── EXPERIMENT: random image points → project to model ────────────────────
num_rand = 12
rand_px  = np.column_stack([
    np.random.uniform(0, w, num_rand),
    np.random.uniform(0, h, num_rand),
])

# draw these as blue dots on the video
shared.sampled_video_pts = [(float(x), float(y)) for x,y in rand_px]

# now project them into model space via H_inv
H_inv = np.linalg.inv(shared.H_video2field)
rand_model = viz.transform_points(rand_px, H_inv)

# draw these as _model‐space_ blue dots on the field
shared.projected_detection_model_pts = [
    (float(mx), float(my)) for mx,my in rand_model
]

# ─── 8) annotate & render ─────────────────────────────────────────────────
viz.video_visualizer.clear_annotations()
viz.field_visualizer.clear_annotations()
viz._annotate_frames(
    viz.video_visualizer.frame,
    viz.field_visualizer.frame
)
out = viz.generate_combined_view()

# ─── 9) display ────────────────────────────────────────────────────────────
display_scale = 0.8  # e.g. 50% of original size
out_resized = cv2.resize(out, None, fx=display_scale, fy=display_scale, interpolation=cv2.INTER_AREA)

cv2.imshow("Random Projection Test", out_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()
